In [8]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# load secrets
load_dotenv()
import math
import json
from IPython.display import JSON

In [9]:
# iex cloud - api data request
iex_key = os.getenv("IEX_API_KEY")
url = 'https://cloud.iexapis.com/stable/ref-data/symbols?token=' + iex_key
response = requests.get(url)
data = response.json()
symbols_df = pd.DataFrame(data)

# NYSE and NASDAQ symbols only (filter)
symbols_df = symbols_df[symbols_df['exchange'].isin(['XNYS', 'XNAS'])]

# select cols
symbols_df = symbols_df[['symbol', 'exchange', 'exchangeName']]

# filter df
print(symbols_df.head())
data_size = len(symbols_df.index)
print()
print(f"Data size: {data_size}") # get length of dataframe

  symbol exchange                 exchangeName
0      A     XNYS  New York Stock Exchange Inc
1     AA     XNYS  New York Stock Exchange Inc
4   AACG     XNAS           Nasdaq All Markets
5   AACI     XNAS           Nasdaq All Markets
6  AACIU     XNAS           Nasdaq All Markets

Data size: 7791


In [10]:
# Function modified for better error handling
def fetch_quote_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data for batch: {symbols_batch}. Status code: {response.status_code}, Response: {response.text}")
        return {}

# Correctly define batch_size
batch_size = 100
total_batches = math.ceil(len(symbols_df) / batch_size)
print(f"Total batches: {total_batches}")
    
# initialize a counter for completed batches
completed_batches = 0

# define the length of the progress bar
progress_bar_length = 50

"""scalable endpoint variable data request"""

quote_data_list = []

for i in range(0, len(symbols_df), batch_size):
    batch_symbols = symbols_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_quote_data(batch_symbols, iex_key)
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        quote_data = batch_data.get(symbol, {}).get('quote', {})
        quote_data_list.append({
            'symbol': symbol, # KEY
            'marketCap': quote_data.get('marketCap', None),
            'latestPrice': quote_data.get('latestPrice', None),
            'close': quote_data.get('close', None), # 'close': 'latestPrice' or 'previousClose
            'previousClose': quote_data.get('previousClose', None),
            'extendedPrice': quote_data.get('extendedPrice', None), 
            'extendedChange': quote_data.get('extendedChange', None), 
            'extendedChangePercent': quote_data.get('extendedChangePercent', None), 
            'latestVolume': quote_data.get('latestVolume', None), # after hours volume
            'volume': quote_data.get('volume', None), # market hours volume
            'previousVolume': quote_data.get('previousVolume', None), # previous day volume
            'iexVolume': quote_data.get('iexVolume', None), # IEX volume
            'primaryExchange': quote_data.get('primaryExchange', None),
            'avgTotalVolume': quote_data.get('avgTotalVolume', None),
            'calculationPrice': quote_data.get('calculationPrice', None),
            'change': quote_data.get('change', None),
            'changePercent': quote_data.get('changePercent', None),
            'companyName': quote_data.get('companyName', None),})
    
    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
quote_df = pd.DataFrame(quote_data_list)


Total batches: 78
Error fetching data for batch: ['A', 'AA', 'AACG', 'AACI', 'AACIU', 'AACT', 'AACT+', 'AACT=', 'AADI', 'AADR', 'AAGR', 'AAL', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPB', 'AAPD', 'AAPL', 'AAPU', 'AAT', 'AAXJ', 'AB', 'ABAT', 'ABBV', 'ABCB', 'ABCL', 'ABCS', 'ABEO', 'ABEV', 'ABG', 'ABIO', 'ABL', 'ABLLL', 'ABLLW', 'ABLV', 'ABLVW', 'ABM', 'ABNB', 'ABOS', 'ABR', 'ABR-D', 'ABR-E', 'ABR-F', 'ABSI', 'ABT', 'ABTS', 'ABUS', 'ABVC', 'ABVX', 'AC', 'ACA', 'ACAB', 'ACABU', 'ACAC', 'ACACU', 'ACACW', 'ACAD', 'ACAH', 'ACAHU', 'ACAHW', 'ACB', 'ACBA', 'ACBAU', 'ACBAW', 'ACCD', 'ACCO', 'ACDC', 'ACEL', 'ACET', 'ACGL', 'ACGLN', 'ACGLO', 'ACHC', 'ACHL', 'ACHR', 'ACHR+', 'ACHV', 'ACI', 'ACIC', 'ACIU', 'ACIW', 'ACLS', 'ACLX', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACNT', 'ACON', 'ACONW', 'ACOR', 'ACP', 'ACP-A', 'ACR', 'ACR-C', 'ACR-D', 'ACRE', 'ACRS']. Status code: 402, Response: Your account does not have the relevant IEX Cloud data bundle add-on for this data. Buy data bundle add-ons at https://iexc

KeyboardInterrupt: 

In [ ]:
quote_df.head(10)

,symbol,marketCap,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,iexVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName
0,A,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,AA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,AACG,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,AACI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,AACIU,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,AACT,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,AACT+,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,AACT=,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,AADI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,AADR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
# Merge with the screener_df
# screener_df = symbols_df.merge(quote_df[['symbol',
#                                         'latestPrice',
#                                         'previousClose', #MOVED UP
#                                         'extendedPrice', #NEW
#                                         'extendedChangePercent', #NEW #IMPORTANT
#                                         'extendedPriceTime', #NEW  
#                                         'latestVolume', #NEW
#                                         'companyName']], on='symbol', how='left')